# Compare offshore gauge results

New comparisons 6/19/25 with 3D results coarsened to 5km (0.05 degree spacing) and Jey's latest poly3d realizations at the same resolution.

All using the 2012 fault model.

Compare to results from earlier:
- https://depts.washington.edu/ptha/CopesHubTsunamis/links/plots_3DvsOkada/

The new results below show much large amplitude tsunamis for both 3D and poly3d sources, still with similar differences between 3D and poly3d.

GeoClaw simulations run to 40 minutes.

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
import os
from scipy.io import loadmat

In [ ]:
CHT = os.environ['CHT']

In [ ]:
outdir3Dg = '%s/geoclaw_runs/offshore_gauges/multirun_3D_5km/gaugedata' % CHT
outdirp3g = '%s/geoclaw_runs/offshore_gauges/multirun_poly3d_5km/gaugedata' % CHT

# with Okada models generated by GeoClaw from Jey's files of 9/24: (not yet plotted)
#outdir1go = '%s/geoclaw_runs/offshore_gauges/multirun_GeoOkada/gaugedata' % CHT

In [ ]:
all_models = \
    ['buried-locking-mur13', 'buried-locking-skl16', 'buried-locking-str10',
     'buried-random-mur13',  'buried-random-skl16',  'buried-random-str10']


models = all_models
events = ['%s-deep' % model for model in models] \
       + ['%s-middle' % model for model in models] \
       + ['%s-shallow' % model for model in models]

events.sort()

events = [e.replace('-','_') for e in events]

#events = events[-2:]  # test on a subset

#events_Okada = [e+'_3D' for e in events]
#events_3D = [e+'_3D' for e in events]

print(events)

In [ ]:
hmax_geoclaw = {}

geoclaw_lat = None

for event in events:
    event_Okada = event + '_poly3d_5km'
    event_3D = event + '_3D_5km'
    geoclaw_gaugemax_path = '%s/offshore_gauges_%s_poly3d_5km_instant_gauges_max.txt' % (outdirp3g, event)
    try:
        geoclaw_gaugemax_Okada = loadtxt(geoclaw_gaugemax_path, skiprows=1)
        if geoclaw_lat is None:
            geoclaw_lat = geoclaw_gaugemax_Okada[:,0]
    except:
        print('Could not read %s' % geoclaw_gaugemax_path)
        geoclaw_gaugemax_Okada = nan*ones((len(geoclaw_lat),2))
    #hmax_geoclaw_Okada.append(geoclaw_gaugemax_Okada[:,1])
    hmax_geoclaw[event_Okada] = geoclaw_gaugemax_Okada[:,1]

    geoclaw_gaugemax_path = '%s/offshore_gauges_%s_3D_5km_instant_gauges_max.txt' % (outdir3Dg, event)
    try:
        geoclaw_gaugemax_3D = loadtxt(geoclaw_gaugemax_path, skiprows=1)
    except:
        print('Could not read %s' % geoclaw_gaugemax_path)
        geoclaw_gaugemax_3D = nan*ones((len(geoclaw_lat),2))
    #hmax_geoclaw_3D.append(geoclaw_gaugemax_3D[:,1])
    hmax_geoclaw[event_3D] = geoclaw_gaugemax_3D[:,1]

In [ ]:
hmax_geoclaw.keys()

In [ ]:
if 0:
    fig,axs = subplots(len(events),1, figsize=(8,3*len(events)))
    for iax,event in enumerate(events):
        event_3D = event + '_3D_5km'
        event_Okada = event + '_poly3d_5km'
        ax = axs[iax]
        ax.plot(geoclaw_lat, hmax_geoclaw[event_3D], 'r', label='GeoClaw 3D')
        ax.plot(geoclaw_lat, hmax_geoclaw[event_Okada], 'k', label='GeoClaw Okada')
        ax.grid(True)
        ax.legend(framealpha=1)
        ax.set_title(event)
    
    tight_layout()
    
    

## MOST results


In [ ]:
#outdir2 = '%s/geoclaw_runs/offshore_gauges/multirun_3DvsOkada/YongsResults' % CHT
#MOST_fname = '%s/3D_vs_Okada.mat' % outdir2
outdir2 = '%s/geoclaw_runs/offshore_gauges/3D_vs_poly3d_5km' % CHT
MOST_fname = '%s/MOST_OffhoreHeight_3d_vs_poly3d.mat' % outdir2
d = loadmat(MOST_fname)

In [ ]:
most_lat = None

hmax_most = {}
events_most = []
for k in range(36):
    Sk = d['S'][k,:]
    if most_lat is None:
        most_lat = Sk[0][2]
    event = str(Sk[0][0][0]).replace('_vertdisp','')
    #print(event)
    #print(Sk[0][3].shape)
    hmax_most[event] = Sk[0][3][:,k]
    #hmax_most.append(Sk[0][3][:,k])
    events_most.append(event)

In [ ]:
hmax_most.keys()

In [ ]:
if 0:
    figure(figsize=(10,6))
    mdata1 = hmax_most['buried_locking_mur13_deep_3D']
    mdata2 = hmax_most['poly3d_uz_resps_source_saved_locking_mur13_deep']
    plot(most_lat, mdata1, 'r')
    plot(most_lat, mdata2, 'k')
    gdata1 = hmax_geoclaw['buried_locking_mur13_deep_3D_5km']
    gdata2 = hmax_geoclaw['buried_locking_mur13_deep_poly3d_5km']
    plot(geoclaw_lat, gdata1, 'm')
    plot(geoclaw_lat, gdata2, 'b')

In [ ]:
plotdir = 'plots_3D_vs_poly3d_5km'
os.system('mkdir -p %s' % plotdir)

for event in events:
    event_3D = event + '_3D'
    event_Okada = event + '_Okada'
    mdata1 = hmax_most['%s_3D' % event]
    mdata2 = hmax_most['poly3d_uz_resps_source_saved_%s' % event.replace('buried_','')]
    gdata1 = hmax_geoclaw['%s_3D_5km' % event]
    gdata2 = hmax_geoclaw['%s_poly3d_5km' % event]
    figure(figsize=(12,4))

    plot(most_lat, mdata1, 'r', label='MOST 3D')
    plot(geoclaw_lat, gdata1, 'm', label='GeoClaw 3D')
    plot(most_lat, mdata2, 'k', label='MOST poly3d')
    plot(geoclaw_lat, gdata2, 'b', label='GeoClaw poly3d')

    xlim(most_lat[-1], most_lat[0])
    grid(True)
    legend()
    title(event);

    if 1:
        fname = '%s/3D_vs_poly3d_offshore_%s.png' % (plotdir, event)
        savefig(fname, bbox_inches='tight')
    

In [ ]:
if 0:
    # OLD
    plotdir = 'plots_3D_vs_poly3d_5km'
    os.system('mkdir -p %s' % plotdir)
    
    for event in events:
        event_3D = event + '_3D'
        event_Okada = event + '_Okada'
        figure(figsize=(12,4))
    
        plot(most_lat, hmax_most[event_3D], 'r', label='MOST 3D')
        plot(geoclaw_lat, hmax_geoclaw[event_3D], 'm', label='GeoClaw 3D')
        plot(most_lat, hmax_most[event_Okada], 'k', label='MOST Okada')
        plot(geoclaw_lat, hmax_geoclaw[event_Okada], 'b', label='GeoClaw Okada')
    
        xlim(most_lat[-1], most_lat[0])
        grid(True)
        legend()
        title(event);
    
        if 1:
            fname = '%s/3DvsOkada_offshore_%s.png' % (plotdir, event)
            savefig(fname, bbox_inches='tight')
    

In [ ]:
plotdir = 'plots_3DvsOkada'
fname = '%s/index.html' % plotdir
index_text = """
<html>
<h1>Tsunami amplitudes using 3D earth structure vs Okada homogeneous half plane</h1>

Computed from Cascadia CoPes Hub Ground Motions using the MOST and GeoClaw tsunami models.
<p>
Static deformation at end of seismic simulation is used for the 3D runs.<br>
Maximum amplitudes are shown at offshore gauges at the 100 m isobath, with 772 gauges from latitude 39 to 48.6.<br>
GeoClaw simulations were run for 40 minutes simulated time<br>

<p>
"""

with open(fname, 'w') as html:
    html.write(index_text)
    for event in events:
        event_3D = event + '_3D'
        event_Okada = event + '_Okada'
        html.write('<p>\n<img src="3DvsOkada_offshore_%s.png" width=800</img>\n' % event)
    html.write('</html>\n')

In [ ]:
len(most_lat), len(geoclaw_lat)